In [1]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import matplotlib
import warnings
warnings.filterwarnings('ignore')

cust_df = pd.read_csv("./santander.csv", encoding='latin-1')
print('dataset shape:', cust_df.shape)
cust_df.head(3)

dataset shape: (76020, 371)


,ID,var3,var15,imp_ent_var16_ult1,imp_op_var39_comer_ult1,imp_op_var39_comer_ult3,imp_op_var40_comer_ult1,imp_op_var40_comer_ult3,imp_op_var40_efect_ult1,imp_op_var40_efect_ult3,...,saldo_medio_var33_hace2,saldo_medio_var33_hace3,saldo_medio_var33_ult1,saldo_medio_var33_ult3,saldo_medio_var44_hace2,saldo_medio_var44_hace3,saldo_medio_var44_ult1,saldo_medio_var44_ult3,var38,TARGET
0,1,2,23,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,39205.17,0
1,3,2,34,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,49278.03,0
2,4,2,23,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,67333.77,0


산탄데르 은행 고객 만족도조사 결과
피처 - 익명화, 컬럼명 익명화, 수치, 0인 데이터가 대다수 - 희소성
타겟 - TARGET

# 데이터 탐색

In [2]:
cust_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 76020 entries, 0 to 76019
Columns: 371 entries, ID to TARGET
dtypes: float64(111), int64(260)
memory usage: 215.2 MB


100명중 4명이 불만족하는 데이터

In [4]:
cust_df['TARGET'].value_counts() #0 만족, 1 불만족
unsatisfied_cnt = cust_df[cust_df['TARGET']==1].TARGET.count()
total_cnt = cust_df.TARGET.count()

print(f'불만족 비율은 : {unsatisfied_cnt/total_cnt:.2f}')

불만족 비율은 : 0.04


최소값, Q1, Q2, Q3 가 0인 피처가 많다.
최소값 -999999.000000 인 피처 var3 > Nan > 중위값/최빈값 2로 대체

In [2]:
cust_df.describe()

,ID,var3,var15,imp_ent_var16_ult1,imp_op_var39_comer_ult1,imp_op_var39_comer_ult3,imp_op_var40_comer_ult1,imp_op_var40_comer_ult3,imp_op_var40_efect_ult1,imp_op_var40_efect_ult3,...,saldo_medio_var33_hace2,saldo_medio_var33_hace3,saldo_medio_var33_ult1,saldo_medio_var33_ult3,saldo_medio_var44_hace2,saldo_medio_var44_hace3,saldo_medio_var44_ult1,saldo_medio_var44_ult3,var38,TARGET
count,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,...,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,7.602000e+04,76020.000000
mean,75964.050723,-1523.199277,33.212865,86.208265,72.363067,119.529632,3.559130,6.472698,0.412946,0.567352,...,7.935824,1.365146,12.215580,8.784074,31.505324,1.858575,76.026165,56.614351,1.172358e+05,0.039569
std,43781.947379,39033.462364,12.956486,1614.757313,339.315831,546.266294,93.155749,153.737066,30.604864,36.513513,...,455.887218,113.959637,783.207399,538.439211,2013.125393,147.786584,4040.337842,2852.579397,1.826646e+05,0.194945
min,1.000000,-999999.000000,5.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,5.163750e+03,0.000000
25%,38104.750000,2.000000,23.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,6.787061e+04,0.000000
50%,76043.000000,2.000000,28.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.064092e+05,0.000000
75%,113748.750000,2.000000,40.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.187563e+05,0.000000
max,151838.000000,238.000000,105.000000,210000.000000,12888.030000,21024.810000,8237.820000,11073.570000,6600.000000,6600.000000,...,50003.880000,20385.720000,138831.630000,91778.730000,438329.220000,24650.010000,681462.900000,397884.300000,2.203474e+07,1.000000


In [6]:
cust_df.var3.value_counts()

 2         74165
 8           138
-999999      116
 9           110
 3           108
           ...  
 231           1
 188           1
 168           1
 135           1
 87            1
Name: var3, Length: 208, dtype: int64

## 피처 선택 - id 삭제, var3 은 최빈값 2로 대체

In [3]:
cust_df.drop('ID', axis=1, inplace=True)
cust_df['var3'].replace(-999999, 2, inplace=True)

cust_df.head()

,var3,var15,imp_ent_var16_ult1,imp_op_var39_comer_ult1,imp_op_var39_comer_ult3,imp_op_var40_comer_ult1,imp_op_var40_comer_ult3,imp_op_var40_efect_ult1,imp_op_var40_efect_ult3,imp_op_var40_ult1,...,saldo_medio_var33_hace2,saldo_medio_var33_hace3,saldo_medio_var33_ult1,saldo_medio_var33_ult3,saldo_medio_var44_hace2,saldo_medio_var44_hace3,saldo_medio_var44_ult1,saldo_medio_var44_ult3,var38,TARGET
0,2,23,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,39205.170000,0
1,2,34,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,49278.030000,0
2,2,23,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,67333.770000,0
3,2,37,0.0,195.0,195.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,64007.970000,0
4,2,39,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,117310.979016,0


## X, y 데이터 분할 


In [4]:
X_features = cust_df.iloc[:, :-1]
y_labels = cust_df.iloc[:, -1]
X_features.shape, y_labels.shape

((76020, 369), (76020,))

## 훈련/테스트 분할

In [5]:
from sklearn.model_selection import train_test_split

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X_features, y_labels, test_size=0.2, random_state=0)
#X_train.count()
print(f'학습 데이터 shape : {X_train.shape}, 테스트 데이터 shape : {X_test.shape}')

train_cnt = y_train.count()
test_cnt = y_test.count()
print(f'학습 데이터 값의 비율 : {y_train.value_counts()/train_cnt}')
print(f'테스트 데이터  값의 비율 : {y_test.value_counts()/test_cnt}')

학습 데이터 shape : (60816, 369), 테스트 데이터 shape : (15204, 369)
학습 데이터 값의 비율 : 0    0.960964
1    0.039036
Name: TARGET, dtype: float64
테스트 데이터  값의 비율 : 0    0.9583
1    0.0417
Name: TARGET, dtype: float64


# DT 예측기

# RF 예측기

# XGBoost 예측기 p271

## 훈련 데이터 > 훈련:검증 분할

In [7]:
X_tr, X_val, y_tr, y_val = train_test_split(X_train, y_train, test_size=0.3, random_state=0)

## XGBoost model

In [18]:
%%time

from xgboost import XGBClassifier
from sklearn.metrics import roc_auc_score

xgb_clf = XGBClassifier(n_estimator=500, learning_rate=0.05, random_state=156)
xgb_clf.fit(X_tr, y_tr, early_stopping_rounds=100, eval_metric='auc', eval_set=[(X_tr, y_tr), (X_val, y_val)])
xgb_roc_score = roc_auc_score(y_test, xgb_clf.predict_proba(X_test)[:, 1])
xgb_roc_score

[0]	validation_0-auc:0.82179	validation_1-auc:0.80068
[1]	validation_0-auc:0.83092	validation_1-auc:0.80941
[2]	validation_0-auc:0.83207	validation_1-auc:0.80903
[3]	validation_0-auc:0.83288	validation_1-auc:0.80889
[4]	validation_0-auc:0.83414	validation_1-auc:0.80924
[5]	validation_0-auc:0.83524	validation_1-auc:0.80907
[6]	validation_0-auc:0.83568	validation_1-auc:0.81005
[7]	validation_0-auc:0.83741	validation_1-auc:0.81088
[8]	validation_0-auc:0.83896	validation_1-auc:0.81305
[9]	validation_0-auc:0.83949	validation_1-auc:0.81363
[10]	validation_0-auc:0.83908	validation_1-auc:0.81277
[11]	validation_0-auc:0.83913	validation_1-auc:0.81260
[12]	validation_0-auc:0.84009	validation_1-auc:0.81325
[13]	validation_0-auc:0.84081	validation_1-auc:0.81329
[14]	validation_0-auc:0.84196	validation_1-auc:0.81380
[15]	validation_0-auc:0.84394	validation_1-auc:0.81540
[16]	validation_0-auc:0.84414	validation_1-auc:0.81573
[17]	validation_0-auc:0.84437	validation_1-auc:0.81577
[18]	validation_0-au

0.8397812474965844

# 하이퍼오피트 파라미터 최적화

In [11]:
from hyperopt import fmin, tpe, Trials, STATUS_OK
from sklearn.base import BaseEstimator, ClassifierMixin
from xgboost import XGBClassifier
from sklearn.model_selection import cross_val_score
from hyperopt  import hp

In [12]:
from hyperopt.pyll.base import scope

# 하이퍼파라미터 검색공간
search_space = { 'n_estimators': scope.int(hp.quniform('n_estimators', 50, 300, 10)), 
                'max_depth': scope.int(hp.quniform('max_depth', 3, 10, 1)),
                'learning_rate': hp.loguniform('learning_rate', np.log(0.01), np.log(0.3)),
                'subsample':hp.uniform('subsample', 0.5, 1.0),
                'colsample_bytree': hp.uniform('colsample_bytree', 0.5, 1.0),
                }

In [15]:
from hyperopt import fmin, tpe, Trials, STATUS_OK
from xgboost import XGBClassifier
from sklearn.model_selection import cross_val_score

def objective_func_xgb(params):
    
    model = XGBClassifier(
        n_estimators=100, #params['n_estimators'],
        max_depth=params['max_depth'],
        learning_rate=params['learning_rate'],
        #subsample=params['subsample'],
        colsample_bytree=params['colsample_bytree'],
        random_state=42,
        eval_metric='logloss'
    )
    #k-fold 추가
    
    score_mean = cross_val_score(model, X_train, y_train, cv=5, scoring='accuracy').mean()
    return {'loss': -1*score_mean , 'status': STATUS_OK}

In [16]:
trials = Trials()

best_params = fmin( 
    fn=objective_func_xgb,
    space=search_space,
    algo=tpe.suggest,
    max_evals=50,
    trials=trials
)

100%|██████████| 50/50 [25:14<00:00, 30.28s/trial, best loss: -0.9610464353768794]


In [17]:
best_params

{'colsample_bytree': 0.5325981930337212,
 'learning_rate': 0.17220434779843125,
 'max_depth': 4.0,
 'n_estimators': 140.0,
 'subsample': 0.7272220988660021}

In [18]:
best_model = XGBClassifier(n_estimators = int(best_params['n_estimators']),
                           max_depth = int(best_params['max_depth']),
                           learning_rate=best_params['learning_rate'],
                           subsample=best_params['subsample'] ,
                           colsample_bytree=best_params['colsample_bytree'],
                           random_state=42,
                           eval_metric='logloss'
                           
                           )

X_tr, X_val, y_tr, y_val = train_test_split(X_train, y_train, test_size=0.3, random_state=0)

In [20]:
eval_list = [(X_tr, y_tr), (X_val, y_val)]

best_model.fit(X_tr, y_tr,
               early_stopping_rounds=100,
               eval_set=eval_list, 
               eval_metric='auc'
               )

[0]	validation_0-auc:0.70170	validation_1-auc:0.69274
[1]	validation_0-auc:0.74094	validation_1-auc:0.72144
[2]	validation_0-auc:0.74818	validation_1-auc:0.72418
[3]	validation_0-auc:0.75586	validation_1-auc:0.73146
[4]	validation_0-auc:0.75838	validation_1-auc:0.73315
[5]	validation_0-auc:0.75989	validation_1-auc:0.73528
[6]	validation_0-auc:0.80785	validation_1-auc:0.78655
[7]	validation_0-auc:0.80728	validation_1-auc:0.78554
[8]	validation_0-auc:0.80845	validation_1-auc:0.78652
[9]	validation_0-auc:0.81935	validation_1-auc:0.79860
[10]	validation_0-auc:0.81766	validation_1-auc:0.79676
[11]	validation_0-auc:0.82507	validation_1-auc:0.80475
[12]	validation_0-auc:0.82458	validation_1-auc:0.80261
[13]	validation_0-auc:0.83249	validation_1-auc:0.81256
[14]	validation_0-auc:0.83041	validation_1-auc:0.80931
[15]	validation_0-auc:0.82875	validation_1-auc:0.80656
[16]	validation_0-auc:0.83599	validation_1-auc:0.81468
[17]	validation_0-auc:0.83712	validation_1-auc:0.81400
[18]	validation_0-au

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.5325981930337212,
              enable_categorical=False, eval_metric='logloss', gamma=0,
              gpu_id=-1, importance_type=None, interaction_constraints='',
              learning_rate=0.17220434779843125, max_delta_step=0, max_depth=4,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=140, n_jobs=12, num_parallel_tree=1,
              predictor='auto', random_state=42, reg_alpha=0, reg_lambda=1,
              scale_pos_weight=1, subsample=0.7272220988660021,
              tree_method='exact', validate_parameters=1, verbosity=None)

In [21]:
from sklearn.metrics import roc_auc_score
xgb_roc_score = roc_auc_score(y_test, best_model.predict_proba(X_test)[:,1])
xgb_roc_score

0.8413232972985847

# 최적화 파라미터 기준 모델 평가

n_estimator=500
learning_rate = 0.15
max_depth = 5


# LGBM 추가
%%time
<!-- 
from xgboost import XGBClassifier
from sklearn.metrics import roc_auc_score

xgb_clf = XGBClassifier(n_estimator=500, learning_rate=0.05, random_state=156)
xgb_clf.fit(X_tr, y_tr, early_stopping_rounds=100, eval_metric='auc', eval_set=[(X_tr, y_tr), (X_val, y_val)])
xgb_roc_score = roc_auc_score(y_test, xgb_clf.predict_proba(X_test)[:, 1])
xgb_roc_score -->

In [ ]:
#XGBM vs LGBM

1. auc 기준 평가 결과
2. 각각의 파라미터값, 모델의 종류

## 특정파라미터러 LGBM 테스트

In [22]:
from lightgbm import LGBMClassifier

lgbm_clf = LGBMClassifier(n_estimators=500, early_stopping_round=50,)
eval_set=[(X_train, y_train), (X_test, y_test)]

lgbm_clf.fit(X_train, y_train, early_stopping_rounds=100, eval_metric="auc", eval_set=eval_set, verbose=True)
lgbm_roc_score = roc_auc_score(y_test, lgbm_clf.predict_proba(X_test)[:,1])
print('ROC AUC: {0:.4f}'.format(lgbm_roc_score))

[1]	training's auc: 0.823612	training's binary_logloss: 0.156016	valid_1's auc: 0.817384	valid_1's binary_logloss: 0.165046
[2]	training's auc: 0.827661	training's binary_logloss: 0.150579	valid_1's auc: 0.818903	valid_1's binary_logloss: 0.160006
[3]	training's auc: 0.837671	training's binary_logloss: 0.146617	valid_1's auc: 0.827707	valid_1's binary_logloss: 0.156323
[4]	training's auc: 0.843171	training's binary_logloss: 0.143452	valid_1's auc: 0.832155	valid_1's binary_logloss: 0.153463
[5]	training's auc: 0.845828	training's binary_logloss: 0.140934	valid_1's auc: 0.834677	valid_1's binary_logloss: 0.151256
[6]	training's auc: 0.846531	training's binary_logloss: 0.138851	valid_1's auc: 0.834093	valid_1's binary_logloss: 0.149427
[7]	training's auc: 0.850239	training's binary_logloss: 0.136968	valid_1's auc: 0.837046	valid_1's binary_logloss: 0.147961
[8]	training's auc: 0.852495	training's binary_logloss: 0.135451	valid_1's auc: 0.837838	valid_1's binary_logloss: 0.146591
[9]	trai

## 하이퍼오피티로 파라미터 탐색

In [27]:
# 하이퍼파라미터 검색공간
search_space = {  'num_leaves': hp.quniform('num_leaves', 32, 64, 1), 
                  'max_depth': hp.quniform('max_depth', 100, 160, 1),
                  'learning_rate': hp.loguniform('learning_rate', np.log(0.01), np.log(0.3)),
                  'subsample':hp.uniform('subsample', 0.5, 1.0),
                  'min_child_samples': hp.quniform('min_child_samples', 60, 100, 1),
                }

In [31]:
from sklearn.model_selection import KFold

def objective_func_lgbm(search_space):
    lgbm_clf =  LGBMClassifier(n_estimators=100, num_leaves=int(search_space['num_leaves']),
                               max_depth=int(search_space['max_depth']),
                               min_child_samples=int(search_space['min_child_samples']), 
                               subsample=search_space['subsample'],
                               learning_rate=search_space['learning_rate'])
    roc_auc_list = []
    kf = KFold(n_splits=3, shuffle=True, random_state=42)
    for tr_idx, val_idx in kf.split(X_train, y_train):
        tr_x, val_x = X_train.iloc[tr_idx], X_train.iloc[val_idx]
        tr_y, val_y = y_train.iloc[tr_idx], y_train.iloc[val_idx]
        lgbm_clf.fit(tr_x, tr_y, eval_set=[(tr_x, tr_y), (val_x, val_y)])
    
        score = roc_auc_score(val_y, lgbm_clf.predict_proba(val_x)[:,1])
        roc_auc_list.append(score)

    #score_mean = cross_val_score(lgbm_clf, X_train, y_train, cv=5, scoring='accuracy').mean()
    return {'loss': -1*np.mean(roc_auc_list) , 'status': STATUS_OK}

In [32]:
from hyperopt import fmin, tpe, Trials, STATUS_OK
trials = Trials()
best = fmin(fn=objective_func_lgbm, space=search_space, algo=tpe.suggest
            , max_evals=50
            , trials=trials
            , rstate=np.random.default_rng(42))

[1]	training's binary_logloss: 0.153535	valid_1's binary_logloss: 0.151809
[2]	training's binary_logloss: 0.146603	valid_1's binary_logloss: 0.146974
[3]	training's binary_logloss: 0.141859	valid_1's binary_logloss: 0.143652
[4]	training's binary_logloss: 0.13847	valid_1's binary_logloss: 0.141402
[5]	training's binary_logloss: 0.135662	valid_1's binary_logloss: 0.139461
[6]	training's binary_logloss: 0.133293	valid_1's binary_logloss: 0.137979
[7]	training's binary_logloss: 0.131363	valid_1's binary_logloss: 0.136974
[8]	training's binary_logloss: 0.129602	valid_1's binary_logloss: 0.136068
[9]	training's binary_logloss: 0.128113	valid_1's binary_logloss: 0.135429
[10]	training's binary_logloss: 0.126626	valid_1's binary_logloss: 0.134844
[11]	training's binary_logloss: 0.125327	valid_1's binary_logloss: 0.134452
[12]	training's binary_logloss: 0.124084	valid_1's binary_logloss: 0.134168
[13]	training's binary_logloss: 0.122937	valid_1's binary_logloss: 0.134024
[14]	training's binary

In [33]:
best

{'learning_rate': 0.036562317848723276,
 'max_depth': 110.0,
 'min_child_samples': 83.0,
 'num_leaves': 34.0,
 'subsample': 0.599418354354013}

In [34]:
best_lgbm_clf =  LGBMClassifier(n_estimators=500, num_leaves=int(best['num_leaves']),
                           max_depth=int(best['max_depth']),
                           min_child_samples=int(best['min_child_samples']), 
                           subsample=round(best['subsample'], 5),
                           learning_rate=round(best['learning_rate'], 5)
                          )

In [36]:
# evaluation metric을 auc로, early stopping은 100 으로 설정하고 학습 수행. 
best_lgbm_clf.fit(X_tr, y_tr, early_stopping_rounds=50, 
            eval_metric="auc",eval_set=[(X_tr, y_tr), (X_val, y_val)])

lgbm_roc_score = roc_auc_score(y_test, best_lgbm_clf.predict_proba(X_test)[:,1])
print('ROC AUC: {0:.4f}'.format(lgbm_roc_score))

[1]	training's auc: 0.828304	training's binary_logloss: 0.161101	valid_1's auc: 0.810496	valid_1's binary_logloss: 0.162483
[2]	training's auc: 0.83048	training's binary_logloss: 0.158243	valid_1's auc: 0.810783	valid_1's binary_logloss: 0.160039
[3]	training's auc: 0.835673	training's binary_logloss: 0.155797	valid_1's auc: 0.814954	valid_1's binary_logloss: 0.157903
[4]	training's auc: 0.83802	training's binary_logloss: 0.1536	valid_1's auc: 0.815181	valid_1's binary_logloss: 0.15613
[5]	training's auc: 0.840339	training's binary_logloss: 0.151675	valid_1's auc: 0.816095	valid_1's binary_logloss: 0.154557
[6]	training's auc: 0.843611	training's binary_logloss: 0.149964	valid_1's auc: 0.816837	valid_1's binary_logloss: 0.153118
[7]	training's auc: 0.844934	training's binary_logloss: 0.148448	valid_1's auc: 0.816977	valid_1's binary_logloss: 0.151846
[8]	training's auc: 0.84899	training's binary_logloss: 0.147014	valid_1's auc: 0.819872	valid_1's binary_logloss: 0.150683
[9]	training's

# XGBM vs. LGBM

#기본 파라미터로 학습 : ROC AUC: 0.8409

#best 파라미터로 학습 : ROC AUC: 0.8419

{'learning_rate': 0.036562317848723276,
 'max_depth': 110.0,
 'min_child_samples': 83.0,
 'num_leaves': 34.0,
 'subsample': 0.599418354354013}

XGBM best 파라미터로 학습 : 0.8413232972985847

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.5325981930337212,
              enable_categorical=False, eval_metric='logloss', gamma=0,
              gpu_id=-1, importance_type=None, interaction_constraints='',
              learning_rate=0.17220434779843125, max_delta_step=0, max_depth=4,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=140, n_jobs=12, num_parallel_tree=1,
              predictor='auto', random_state=42, reg_alpha=0, reg_lambda=1,
              scale_pos_weight=1, subsample=0.7272220988660021,
              tree_method='exact', validate_parameters=1, verbosity=None)